In [19]:
%load_ext autoreload
# reload modules automatically before each cell
%autoreload 2

In [20]:
from pyiron_workflow import Workflow as wf

In [21]:
import requests
import os
os.environ["REQUESTS_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt" # if not already set by the OS; doesn't hurt

# fixes a bug in SPARQLwrapper
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# importing pandas to make printed dataframes prettier
import pandas as pd, hashlib
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.width', 200)

from pmd_demo_tools import mesh_tools, sparql_tools
from pmd_demo_tools.query_collection import S355queries
S355queries = S355queries()

import matplotlib.pyplot as plt
import json

In [23]:
def populate_mesh_registry(token_file_path: str | None) -> mesh_tools.RecursiveNamespace:
    # read in all servers on the mesh
    partners = mesh_tools.mesh_namespace_grouped_by_company()

    # attach hosted services
    _ = mesh_tools.attach_services_in_place(partners)
    
    if token_file_path is not None:
        with open(token_file_path, "r", encoding="utf-8") as f:
            tokens = json.load(f, object_hook=mesh_tools.namespace_object_hook())

        mesh_tools.attach_tokens_to_partners(partners, tokens, overwrite=True, warn_missing=False)

    return partners

/home/jovyan/work/pmd_mesh-demonstrator/pmd_demo_tools/pmd_demo_tools/mesh_tools.py:606: UserWarning: Failed fetching services for glassomer.pmd.internal: 500 Server Error: Internal Server Error for url: http://mesh-listing.c.pmd.internal/api/v1/pmds/glassomer.pmd.internal.
  warnings.warn(f"Failed fetching services for {zone}: {e}.", category=UserWarning)


In [29]:
def reduce_registry(partners: mesh_tools.RecursiveNamespace,
                    selection: list[str]) -> mesh_tools.RecursiveNamespace:
    partners = mesh_tools.select_toplevel(partners, selection, deepcopy=True)
    return partners

In [31]:
def query_datasets_on_mesh(partners: mesh_tools.RecursiveNamespace,
                      query: sparql_tools.SparqlQuery,
                      datasets: list[str] | None = None,
                      print_to_screen: bool = False) -> mesh_tools.RecursiveNamespace:
    results = sparql_tools.federated_query(partners=partners, datasets=datasets, query=query.query, columns=query.headers, print_to_screen=print_to_screen)
    results_rns = mesh_tools.RecursiveNamespace(**orientation_results)
    return results_rns